<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

# changements de variable

ce notebook élabore autour du projet numérique "tracé du contour"

## rappel

dans le projet numérique qui vise à calculer les lignes de niveau d'une fonction, on vous suggère

* de traiter avec `contour_simple` le cas très spécifique 
  * d'une fonction définie sur le pavé unité $[0..1]^2$,
  * et avec une courbe qui entre dans le pavé par le bord gauche (x=0)
* puis de généraliser avec `contour` en découpant un domaine quelconque en pavés élémentaires

## notre sujet

Nous allons étudier les transformations qui permettent de passer du cas général au cas spécifique.

On se donne donc

$f : [x1 .. x2] \times [y1 .. y2] \longrightarrow \mathbb{R}$

et pour se ramener à une fonction qui vérifie les hypothèses de `contour_simple`, on voit qu'il s'agit de déformer le domaine de manière linéaire de sorte que le pavé $[x1 .. x2] \times [y1 .. y2]$ corresponde à $[0..1]^2$.

De plus, puisque `contour_simple` fait l'hypothèse supplémentaire que le contour commence sur le bord gauche, on va envisager également les 4 rotations du pavé.

## comment s'y prendre

on commence par coder les changements de variable élémentaires qui correspondent 

* aux translations
* aux homothéties (scaling)
* aux rotations autour du point (0.5, 0.5)

### translations

le changement de variable le plus élémentaire est la translation:

en partant de la fonction $f : \mathbb{R}^2 \longrightarrow \mathbb{R}$ et étant donné deux réels $(x_0,y_0)$, on peut facilement dériver une fonction $f_{t(x_0,y_0)}$ qui correspond à $f$ translatée de $(x_0, y_0)$ par

$f_{t(x_0,y_0)}(x, y) = f(x-x_0, y-y_0)$

on peut très facilement coder la fonction `translate` en Python :

In [ ]:
# en entrée f est une fonction R2->R
def translate(f, x0, y0):
    def translated(x, y):
        return f(x-x0, y-y0)
    # en sortie on veut aussi une fonction R2->R
    return translated

si on voulait décrire formellement la fonction translate, on écrirait

$translate: {\mathbb{R}}^{\mathbb{R}^2} \times \mathbb{R}^2\longrightarrow {\mathbb{R}}^{\mathbb{R}^2}$

qui est une façon très pédante de dire le fait très simple que

 
* si on a en entrée
  * une fonction $f: \mathbb{R}^2 \longrightarrow \mathbb{R}$,
  * et deux réels
* alors on aura en sortie
  * une nouvelle fonction $\mathbb{R}^2 \longrightarrow \mathbb{R}$

on peut écrire exactement la même chose en utilisant une lambda

In [ ]:
def translate(f, x0, y0):
    def translated(x, y):
        return f(x-x0, y-y0)
    return translated

In [ ]:
# c'est totalement équivalent 
# d'écrire
def translate(f, x0, y0):
    return lambda x, y: f(x-x0, y-y0)

#### visualisation

on va visualiser une fonction disons reconnaissable

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

In [ ]:
def gaussian(x, y):
    """
    centered gaussian on X, Y
    not normalized,
    i.e. gaussian(0, 0) = 1
    """
    return np.exp(-x**2 -y**2)

In [ ]:
def h(x, y):
    return 2*(gaussian(x, y) 
              - gaussian(x-1, y-1))

In [ ]:
# le maximum est ici
h(0, 0)

In [ ]:
# le domaine de visualisation
X = np.arange(-4, 4, 0.25)
Y = np.arange(-4, 4, 0.25)
X, Y = np.meshgrid(X, Y)

In [ ]:
def show_3d(X, Y, Z, sizex=6, sizey=6):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.gca(projection='3d')
    ax.plot_surface(
        X, Y, Z, cmap=cm.coolwarm,
        linewidth=0, antialiased=False);

In [ ]:
Z = h(X, Y)
show_3d(X, Y, Z)

In [ ]:
# on fabrique la fonction translatée
translated_h = translate(h, 1, 2)

In [ ]:
# le maximum doit être ici
translated_h(1, 2)

In [ ]:
Z2 = translated_h(X, Y)

show_3d(X, Y, Z2)

### changement d'échelle

on s'y prend de manière similaire pour les déformations d'étirement

In [ ]:
def scale(f, sx, sy):
    def scaled(x, y):
        return f(x/sx, y/sy)
    return scaled

In [ ]:
def scale(f, sx, sy):
    return lambda x, y: f(x/sx, y/sy)

In [ ]:
scaled_h = scale(h, 3, 2)

In [ ]:
# maximum inchangé
scaled_h(0, 0)

In [ ]:
Z3 = scaled_h(X, Y)

show_3d(X, Y, Z3)

### rotations

In [ ]:
from enum import Enum

class Angle(Enum):
    LEFT = 0      # no rotation
    TOP = 1       # rotate π/2 counter-clockwise around (0.5, 0.5)
    RIGHT = 2     # rotate π around around (0.5, 0.5)
    BOTTOM = 3    # rotate π/2 clockwise around (0.5, 0.5)

def rotate(f, angle: Angle):
    if angle == Angle.LEFT:
        return f
    if angle == Angle.TOP:
        return lambda x, y: f(1-y, x)
    if angle == Angle.RIGHT:
        return lambda x, y: f(1-x, 1-y)
    if angle == Angle.BOTTOM:
        return lambda x, y: f(y, 1-x)

In [ ]:
# on focalise sur le carré unité
X1 = np.arange(0, 1, .1)
Y1 = np.arange(0, 1, .1)
X1, Y1 = np.meshgrid(X1, Y1)

Zh = h(X1, Y1)

In [ ]:
show_3d(X1, Y1, Zh)

In [ ]:
# identique à h en principe
h_l = rotate(h, Angle.LEFT)

Zl = h_l(X1, Y1)

In [ ]:
# tourné d'un quart
h_t = rotate(h, Angle.TOP)

Zt = h_t(X1, Y1)

In [ ]:
show_3d(X1, Y1, Zl)

In [ ]:
show_3d(X1, Y1, Zt)

In [ ]:
h_r = rotate(h, Angle.RIGHT)

Zr = h_r(X1, Y1)

In [ ]:
h_b = rotate(h, Angle.BOTTOM)

Zb = h_b(X1, Y1)

In [ ]:
show_3d(X1, Y1, Zr)

In [ ]:
show_3d(X1, Y1, Zb)